In [4]:
import numpy as np 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import control.matlab as control_
import control
import math 
from sympy import symbols, solve

def pos(lst):
    return [x for x in lst if x > 0] or None
def jitter(inps):
    BW,PM= inps
    pi=math.pi
    X= symbols('x')
    #Assigning some constants 
    Fref=96e6
    Tref=1/Fref
    Fo=5e9
    N=Fo/Fref
    KVCO=150e6*2*pi
    BW=2*pi*BW
    PM=(PM/180)*pi
    delta=1e-12
    A=(((BW)**2)*2*pi*delta*N)/(KVCO*math.sqrt(1+(math.tan(PM)**2)))
    B=(math.tan(PM)*Fref*BW*2*pi*delta*N)/(KVCO*(math.sqrt(1+(math.tan(PM)**2))))

    # TDC noise specturm 
    T_TDC=(4*(pi**2)*(delta**2))/(12*Tref)

    #VCO phase noise 
    w=np.linspace(2*pi*12e3,2*pi*20e6,5000)
    T_osc=control.tf([1,2*pi*10e4],[1,0])
    T_osc=T_osc*30
    s=control.tf([1],[1,0,0])
    T_osc=T_osc*s
    T_osc

    #VCO phase noise 
    (mag, phase_rad, w) = control.bode(T_osc, w,dB=True,Hz=True,plot=False)
    '''for i in range(np.shape(w)[0]):
        if 10*np.log10(mag[i]) <= -125 :
            mag[i:]=mag[i]
            break'''
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('VCO phase noise')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    # Divider transfer phase noise 
    T_Divider=control.tf([1,2*300e3*pi],[1,0])
    T_Divider=T_Divider*3.01506e-18

    (mag_Divider, phase_rad_Divider, w) = control.bode(T_Divider, w,dB=True,Hz=True,plot=False)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_Divider), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('Dividers phase noise')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    #Reference phase noise 
    T_ref=control.tf([1,1e3*2*pi],[1,0])
    T_ref_1=control.tf([1,1e4*pi],[1])
    T_ref_2=control.tf([1,1e7*2*pi],[1])
    T_ref=T_ref*(2.224970786e-19)*T_ref_1*T_ref_2
    s=control.tf([1],[1,0,0])
    T_ref=T_ref*s
    T_ref

    #Reference phase noise 
    (mag_ref, phase_rad_ref, w) = control.bode(T_ref, w,dB=True,Hz=True,plot=False)
    for i in range(np.shape(w)[0]):
        if 10*np.log10(mag_ref[i]) <= -220 :
            mag_ref[i:]=mag_ref[i]
            break 
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_ref), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('Reference phase noise')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    # # Feedback of the loop in the middle 
    # #Closed loop 
    # Bs_=(Ts)/(1+(Ts))
    # (mag_FB, phase_rad_FB, w) = control.bode_plot(Bs, w,dB=True, deg= True )

    # open gain for Feedback loop 
    Ts=control.tf([B*Tref,A],[2*pi*N*delta,0,0])
    Ts=Ts*KVCO
    (mag_Feed, phase_rad_Feed, w) = control.bode_plot(Ts, w,dB=True, deg= True ,Hz=True, margins=True,plot=False)


    # the overall closed loop
    Bs=(Ts*N)/((1+(Ts)))
    (mag_FB, phase_rad_FB, w) = control.bode_plot(Bs, w,dB=True, Hz=True,deg= True ,plot=False)


    # TDC transfer specturm
    T_out_TDC= Bs*(2*pi*delta/Tref)
    #T_out_TDC=T_out_TDC/(1+(T_out_TDC/(N*(2*pi*delta/Tref))))
    (mag_out_TDC, phase_out_TDC, w) = control.bode_plot(T_out_TDC, w,dB=True, deg= True,plot=False)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_TDC)**2), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('(phase noise TDC / output)\u00b2')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    TDC_noise_output=T_TDC*((mag_out_TDC)**2)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(TDC_noise_output), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('TDC noise at output')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    #Transfer function of different blocks 
    #Transfer function of VCO phase output to phase of VCO
    T_out_VCO=1/((1+(Ts)))
    (mag_out_VCO, phase_out_VCO, w) = control.bode_plot(T_out_VCO, w,dB=True, deg= True,plot=False)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_VCO)**2), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('(phase noise VCO / output)\u00b2')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    VCO_noise_output=mag*((mag_out_VCO)**2)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(VCO_noise_output), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('VCO noise at output')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    #Transfer function of different blocks 
    #Transfer function of CP phase output to phase of Reference
    T_out_ref=Bs
    (mag_out_ref, phase_out_ref, w) = control.bode_plot(T_out_ref, w,dB=True, deg= True,plot=False)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_ref)**2), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('(Ref phase noise/ output)\u00b2')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    ref_noise_output=mag_ref*((mag_out_ref)**2)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(ref_noise_output), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('ref noise at output')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    #Transfer function of N_Divider phase output to phase of Reference

    T_out_N_Divider=Bs
    (mag_out_N_Divider, phase_out_N_Divider, w) = control.bode_plot(T_out_N_Divider, w,dB=True, deg= True,plot=False)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_N_Divider)**2), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('N_Divider phase noise/ output)\u00b2')
    plt.grid() 
    plt.legend(labels=('mag',))
    '''
    N_divider_noise_output=((mag_out_N_Divider)**2)*mag_Divider
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(N_divider_noise_output), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('N Divider noise at output')
    plt.grid() 
    plt.legend(labels=('mag',))
    '''
    #calculation every noise at the output 
    noise_output=VCO_noise_output+ref_noise_output+N_divider_noise_output+(mag_Divider)
    '''plt.plot(np.log10(w/(2*pi)), 10*np.log10(noise_output), 'blue') 
    plt.xlabel('Freq [Hz]') 
    plt.ylabel('dBc/HZ') 
    plt.title('noise at output')
    plt.grid() 
    plt.legend(labels=('mag',))'''

    jitter= np.trapz(noise_output, x=w/(2*pi))
    jitter=jitter*2
    jitter=(math.sqrt(jitter))/(2*pi*Fo)
    return(jitter*1e15)

In [5]:
jitter([1e6, 7.50000000e+01])

50.7970829075034

In [7]:
import scipy.optimize as spo
bnds=((1e6,9.6e6),(60,80))
intials=[1e6,60]
res=spo.minimize(jitter,intials,method='Nelder-Mead',bounds=bnds)

In [8]:
res

 final_simplex: (array([[9.6e+06, 8.0e+01],
       [9.6e+06, 8.0e+01],
       [9.6e+06, 8.0e+01]]), array([22.58126028, 22.58126028, 22.58126028]))
           fun: 22.581260282730618
       message: 'Optimization terminated successfully.'
          nfev: 30
           nit: 15
        status: 0
       success: True
             x: array([9.6e+06, 8.0e+01])